Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
image_size = 28
num_labels = 10
batch_size=128 # 随机梯度下降时候的数据大小
hidden_layer_nodes=1024
decay_rate=0.01

graph=tf.Graph()
with graph.as_default():
    train_data_set=tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    train_data_label=tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    # 对验证集和测试集做常值的定义
    validation_data_set=tf.constant(valid_dataset)
    test_data_set=tf.constant(test_dataset)
    # 定义随机初始化隐藏层的权重和偏执项
    hidden_weights=tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer_nodes]))
    hidden_biases=tf.Variable(tf.zeros([hidden_layer_nodes]))
    hidden_layers=tf.nn.relu(tf.matmul(train_data_set,hidden_weights)+hidden_biases)
    
    # 设置全连接层
    out_weights=tf.Variable(tf.truncated_normal([hidden_layer_nodes,num_labels]))
    out_biases=tf.Variable(tf.zeros([num_labels]))
    out_layers=tf.matmul(hidden_layers,out_weights)+out_biases
    
    # 设置损失函数+对隐含层和输出层的权重/偏执项都加上l2惩罚
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=out_layers,labels=train_data_label)) +\
                        decay_rate*tf.nn.l2_loss(hidden_weights) +\
                        decay_rate*tf.nn.l2_loss(hidden_biases) +\
                        decay_rate*tf.nn.l2_loss(out_weights) +\
                        decay_rate*tf.nn.l2_loss(out_biases)

    optimiser=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction=tf.nn.softmax(out_layers)
    # 预测检验数据集
    valid_hidden_layer=tf.nn.relu(tf.matmul(validation_data_set,hidden_weights)+hidden_biases)
    valid_prediction=tf.nn.softmax(tf.matmul(valid_hidden_layer,out_weights)+out_biases)
    # 预测测试数据集
    test_hidden_layer=tf.nn.relu(tf.matmul(test_data_set,hidden_weights)+hidden_biases)
    test_prediction=tf.nn.softmax(tf.matmul(test_hidden_layer,out_weights)+out_biases)
    
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
# 训练模型
epoch_steps=3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for epoch in range(epoch_steps):
        offset=(epoch*batch_size) % (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={train_data_set:batch_data,train_data_label:batch_labels}
        _,l,predictions=sess.run([optimiser,loss,train_prediction],feed_dict=feed_dict)
        if epoch % 500 ==0:
            print('mini batch loss at step :%d is: %0.2f' % (epoch,l))
            print('minibatch accuracy: %0.2f%%' % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

mini batch loss at step :0 is: 3525.47
minibatch accuracy: 6.25%
Validation accuracy: 30.6%
mini batch loss at step :500 is: 21.33
minibatch accuracy: 78.12%
Validation accuracy: 84.9%
mini batch loss at step :1000 is: 0.86
minibatch accuracy: 81.25%
Validation accuracy: 83.6%
mini batch loss at step :1500 is: 0.61
minibatch accuracy: 87.50%
Validation accuracy: 84.0%
mini batch loss at step :2000 is: 0.73
minibatch accuracy: 83.59%
Validation accuracy: 83.2%
mini batch loss at step :2500 is: 0.63
minibatch accuracy: 86.72%
Validation accuracy: 83.9%
mini batch loss at step :3000 is: 0.71
minibatch accuracy: 82.81%
Validation accuracy: 83.6%
Test accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

ANS:Few batches make accuracy on training data unstable, as well as make validated data unstable.

In [9]:
image_size = 28
num_labels = 10
batch_size=80 # 随机梯度下降时候的数据大小
hidden_layer_nodes=1024
decay_rate=0.01

graph=tf.Graph()
with graph.as_default():
    train_data_set=tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    train_data_label=tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    # 对验证集和测试集做常值的定义
    validation_data_set=tf.constant(valid_dataset)
    test_data_set=tf.constant(test_dataset)
    # 定义随机初始化隐藏层的权重和偏执项
    hidden_weights=tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer_nodes]))
    hidden_biases=tf.Variable(tf.zeros([hidden_layer_nodes]))
    hidden_layers=tf.nn.relu(tf.matmul(train_data_set,hidden_weights)+hidden_biases)
    
    # 设置全连接层
    out_weights=tf.Variable(tf.truncated_normal([hidden_layer_nodes,num_labels]))
    out_biases=tf.Variable(tf.zeros([num_labels]))
    out_layers=tf.matmul(hidden_layers,out_weights)+out_biases
    
    # 设置损失函数+对隐含层和输出层的权重/偏执项都加上l2惩罚
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=out_layers,labels=train_data_label)) +\
                        decay_rate*tf.nn.l2_loss(hidden_weights) +\
                        decay_rate*tf.nn.l2_loss(hidden_biases) +\
                        decay_rate*tf.nn.l2_loss(out_weights) +\
                        decay_rate*tf.nn.l2_loss(out_biases)

    optimiser=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction=tf.nn.softmax(out_layers)
    # 预测检验数据集
    valid_hidden_layer=tf.nn.relu(tf.matmul(validation_data_set,hidden_weights)+hidden_biases)
    valid_prediction=tf.nn.softmax(tf.matmul(valid_hidden_layer,out_weights)+out_biases)
    # 预测测试数据集
    test_hidden_layer=tf.nn.relu(tf.matmul(test_data_set,hidden_weights)+hidden_biases)
    test_prediction=tf.nn.softmax(tf.matmul(test_hidden_layer,out_weights)+out_biases)
    
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
# 训练模型
epoch_steps=3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for epoch in range(epoch_steps):
        offset=(epoch*batch_size) % (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={train_data_set:batch_data,train_data_label:batch_labels}
        _,l,predictions=sess.run([optimiser,loss,train_prediction],feed_dict=feed_dict)
        if epoch % 500 ==0:
            print('mini batch loss at step :%d is: %0.2f' % (epoch,l))
            print('minibatch accuracy: %0.2f%%' % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

mini batch loss at step :0 is: 3545.77
minibatch accuracy: 13.75%
Validation accuracy: 25.3%
mini batch loss at step :500 is: 21.25
minibatch accuracy: 87.50%
Validation accuracy: 83.4%
mini batch loss at step :1000 is: 1.10
minibatch accuracy: 75.00%
Validation accuracy: 83.3%
mini batch loss at step :1500 is: 0.82
minibatch accuracy: 82.50%
Validation accuracy: 83.5%
mini batch loss at step :2000 is: 0.77
minibatch accuracy: 80.00%
Validation accuracy: 82.2%
mini batch loss at step :2500 is: 0.80
minibatch accuracy: 80.00%
Validation accuracy: 82.3%
mini batch loss at step :3000 is: 0.72
minibatch accuracy: 85.00%
Validation accuracy: 82.8%
Test accuracy: 89.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [15]:
image_size = 28
num_labels = 10
batch_size=128 # 随机梯度下降时候的数据大小
hidden_layer_nodes=1024
decay_rate=0.01
keep_prob=0.6

graph=tf.Graph()
with graph.as_default():
    train_data_set=tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    train_data_label=tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    # 对验证集和测试集做常值的定义
    validation_data_set=tf.constant(valid_dataset)
    test_data_set=tf.constant(test_dataset)
    # 定义随机初始化隐藏层的权重和偏执项
    hidden_weights=tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer_nodes]))
    hidden_biases=tf.Variable(tf.zeros([hidden_layer_nodes]))
    hidden_layers=tf.nn.relu(tf.matmul(train_data_set,hidden_weights)+hidden_biases)
    # 引入dropout机制
    dropout_layer=tf.nn.dropout(hidden_layers,rate=1-keep_prob)
    
    # 设置全连接层
    out_weights=tf.Variable(tf.truncated_normal([hidden_layer_nodes,num_labels]))
    out_biases=tf.Variable(tf.zeros([num_labels]))
    out_layers=tf.matmul(hidden_layers,out_weights)+out_biases
    
    # 设置损失函数+对隐含层和输出层的权重/偏执项都加上l2惩罚
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=out_layers,labels=train_data_label)) +\
                        decay_rate*tf.nn.l2_loss(hidden_weights) +\
                        decay_rate*tf.nn.l2_loss(hidden_biases) +\
                        decay_rate*tf.nn.l2_loss(out_weights) +\
                        decay_rate*tf.nn.l2_loss(out_biases)

    optimiser=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction=tf.nn.softmax(out_layers)
    # 预测检验数据集
    valid_hidden_layer=tf.nn.relu(tf.matmul(validation_data_set,hidden_weights)+hidden_biases)
    valid_prediction=tf.nn.softmax(tf.matmul(valid_hidden_layer,out_weights)+out_biases)
    # 预测测试数据集
    test_hidden_layer=tf.nn.relu(tf.matmul(test_data_set,hidden_weights)+hidden_biases)
    test_prediction=tf.nn.softmax(tf.matmul(test_hidden_layer,out_weights)+out_biases)
    
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
# 训练模型
epoch_steps=3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for epoch in range(epoch_steps):
        offset=(epoch*batch_size) % (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={train_data_set:batch_data,train_data_label:batch_labels}
        _,l,predictions=sess.run([optimiser,loss,train_prediction],feed_dict=feed_dict)
        if epoch % 500 ==0:
            print('mini batch loss at step :%d is: %0.2f' % (epoch,l))
            print('minibatch accuracy: %0.2f%%' % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

mini batch loss at step :0 is: 3525.36
minibatch accuracy: 9.38%
Validation accuracy: 26.1%
mini batch loss at step :500 is: 21.37
minibatch accuracy: 81.25%
Validation accuracy: 84.2%
mini batch loss at step :1000 is: 0.87
minibatch accuracy: 81.25%
Validation accuracy: 83.5%
mini batch loss at step :1500 is: 0.61
minibatch accuracy: 86.72%
Validation accuracy: 83.9%
mini batch loss at step :2000 is: 0.73
minibatch accuracy: 82.81%
Validation accuracy: 83.3%
mini batch loss at step :2500 is: 0.62
minibatch accuracy: 86.72%
Validation accuracy: 84.0%
mini batch loss at step :3000 is: 0.71
minibatch accuracy: 83.59%
Validation accuracy: 83.5%
Test accuracy: 90.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [32]:
image_size = 28
num_labels = 10
batch_size=128 # 随机梯度下降时候的数据大小
hidden_layer_nodes=1024
decay_rate=0.01
keep_prob=0.6
learning_rate=0.5

graph=tf.Graph()
with graph.as_default():
    train_data_set=tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    train_data_label=tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    # 对验证集和测试集做常值的定义
    validation_data_set=tf.constant(valid_dataset)
    test_data_set=tf.constant(test_dataset)
    # 定义随机初始化隐藏层的权重和偏执项
    hidden_weights=tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer_nodes]))
    hidden_biases=tf.Variable(tf.zeros([hidden_layer_nodes]))
    hidden_layers=tf.nn.relu(tf.matmul(train_data_set,hidden_weights)+hidden_biases)
    # 引入dropout机制
    dropout_layer=tf.nn.dropout(hidden_layers,rate=1-keep_prob)
    
    # 设置全连接层
    out_weights=tf.Variable(tf.truncated_normal([hidden_layer_nodes,num_labels]))
    out_biases=tf.Variable(tf.zeros([num_labels]))
    out_layers=tf.matmul(hidden_layers,out_weights)+out_biases
    
    # 设置损失函数+对隐含层和输出层的权重/偏执项都加上l2惩罚
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=out_layers,labels=train_data_label)) +\
                        decay_rate*tf.nn.l2_loss(hidden_weights) +\
                        decay_rate*tf.nn.l2_loss(hidden_biases) +\
                        decay_rate*tf.nn.l2_loss(out_weights) +\
                        decay_rate*tf.nn.l2_loss(out_biases)

    # 引入学习率
    global_step=tf.Variable(0,trainable=False)
    learning_rate = tf.train.exponential_decay(learning_rate,global_step,500,0.96)
    optimiser=tf.train.GradientDescentOptimizer(0.5).minimize(loss,global_step=global_step)
    train_prediction=tf.nn.softmax(out_layers)
    # 预测检验数据集
    valid_hidden_layer=tf.nn.relu(tf.matmul(validation_data_set,hidden_weights)+hidden_biases)
    valid_prediction=tf.nn.softmax(tf.matmul(valid_hidden_layer,out_weights)+out_biases)
    # 预测测试数据集
    test_hidden_layer=tf.nn.relu(tf.matmul(test_data_set,hidden_weights)+hidden_biases)
    test_prediction=tf.nn.softmax(tf.matmul(test_hidden_layer,out_weights)+out_biases)
    
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
# 训练模型
epoch_steps=3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for epoch in range(epoch_steps):
        offset=(epoch*batch_size) % (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={train_data_set:batch_data,train_data_label:batch_labels}
        _,l,predictions=sess.run([optimiser,loss,train_prediction],feed_dict=feed_dict)
        if epoch % 500 ==0:
            print('mini batch loss at step :%d is: %0.2f' % (epoch,l))
            print('minibatch accuracy: %0.2f%%' % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

mini batch loss at step :0 is: 3406.80
minibatch accuracy: 17.19%
Validation accuracy: 33.8%
mini batch loss at step :500 is: 21.35
minibatch accuracy: 80.47%
Validation accuracy: 84.6%
mini batch loss at step :1000 is: 0.87
minibatch accuracy: 80.47%
Validation accuracy: 83.5%
mini batch loss at step :1500 is: 0.60
minibatch accuracy: 87.50%
Validation accuracy: 83.9%
mini batch loss at step :2000 is: 0.73
minibatch accuracy: 82.81%
Validation accuracy: 83.3%
mini batch loss at step :2500 is: 0.63
minibatch accuracy: 85.94%
Validation accuracy: 84.0%
mini batch loss at step :3000 is: 0.71
minibatch accuracy: 82.81%
Validation accuracy: 83.6%
Test accuracy: 90.3%


In [26]:
help(tf.train.exponential_decay)

Help on function exponential_decay in module tensorflow.python.training.learning_rate_decay:

exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase=False, name=None)
    Applies exponential decay to the learning rate.
    
    When training a model, it is often recommended to lower the learning rate as
    the training progresses.  This function applies an exponential decay function
    to a provided initial learning rate.  It requires a `global_step` value to
    compute the decayed learning rate.  You can just pass a TensorFlow variable
    that you increment at each training step.
    
    The function returns the decayed learning rate.  It is computed as:
    
    ```python
    decayed_learning_rate = learning_rate *
                            decay_rate ^ (global_step / decay_steps)
    ```
    
    If the argument `staircase` is `True`, then `global_step / decay_steps` is an
    integer division and the decayed learning rate follows a staircase functio